# Interpolate to regular UPS grid
- convert lat-lon to UPS (done)
- interpolate to a regular grid in UPS
- export to geotiff

This is a work in progress. 

In [ ]:
import xarray as xr
import hvplot.xarray
import hvplot as hv

In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jkingslake/proxy/8787/status,
Dashboard: /user/jkingslake/proxy/8787/status,Workers: 2
Total threads: 2,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33925,Workers: 2
Dashboard: /user/jkingslake/proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:36941,Total threads: 1
Dashboard: /user/jkingslake/proxy/38373/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:46443,


In [6]:
racmo_AP = xr.open_dataset('gs://ldeo-glaciology/RACMO/JK_tests/all_vars_full_1' , engine='zarr', consolidated=True, chunks={}) 

In [7]:
racmo_AP = racmo_AP.squeeze()
racmo_AP

<xarray.Dataset>
Dimensions:   (rlat: 454, rlon: 501, time: 14853)
Coordinates:
    height    float64 0.0
    lat       (rlat, rlon) float64 dask.array<chunksize=(454, 501), meta=np.ndarray>
    lon       (rlat, rlon) float64 dask.array<chunksize=(454, 501), meta=np.ndarray>
  * rlat      (rlat) float64 -8.7 -8.65 -8.6 -8.55 -8.5 ... 5.9 5.9 5.95 5.95
  * rlon      (rlon) float64 -32.0 -31.95 -31.9 -31.85 ... -12.6 -12.55 -12.55
  * time      (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-08-31
Data variables:
    precip    (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>
    smb       (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>
    snowfall  (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>
    snowmelt  (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>
    subl      (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>
    t2m       (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>
    tskin     (time, rlat, rlon) float32 dask.array<chunksize=(2476, 76, 84), meta=np.ndarray>

In [8]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:44137")
client

<Client: 'tcp://127.0.0.1:44137' processes=2 threads=2, memory=16.00 GiB>

In [6]:
racmo_AP.precip.isel(time=0, height=0).hvplot()

:Image   [rlon,rlat]   (precip)

In [5]:
racmo_AP.rlat[:-1].values-racmo_AP.rlat[1:].values

array([-5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -8.88178420e-16, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -5.00000000e-02, -8.88178420e-16, -5.00000000e-02,
       -8.88178420e-16, -5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
       -5.00000000e-02, -8.88178420e-16, -5.00000000e-02, -8.88178420e-16,
       -5.00000000e-02, -

In [9]:
import pyproj
source_crs = 'epsg:4326' # Global lat-lon coordinate system
target_crs = 'epsg:3031' # Coordinate system of the file

latlon_to_polar= pyproj.Transformer.from_crs(source_crs, target_crs)

In [10]:
X,Y = latlon_to_polar.transform(racmo_AP.lat, racmo_AP.lon)

racmo_AP = racmo_AP.assign_coords(x=(['rlat', 'rlon'], X), y=(['rlat', 'rlon'], Y))

In [11]:
import matplotlib.pyplot as plt

In [ ]:
temp = racmo_AP.t2m.isel(time=0)
racmo_AP.hvplot.scatter(x='x', y='y')

In [10]:
racmo_AP.lat.values

array([[-56.96002341, -57.00805813, -57.05608767, ..., -74.7283254 ,
                 nan, -74.76923224],
       [-56.97175536, -57.01981164, -57.06786278, ..., -74.75629254,
                 nan, -74.7972783 ],
       [-56.98342386, -57.03150159, -57.07957424, ..., -74.78414945,
                 nan, -74.82521412],
       ...,
       [-57.51753347, -57.56660777, -57.61567954, ..., -76.10731695,
                 nan, -76.15248691],
       [-57.50938407, -57.55844294, -57.60749925, ..., -76.08635395,
                 nan, -76.13145288],
       [         nan,          nan,          nan, ...,          nan,
                 nan,          nan]])

In [12]:
import numpy as np
np.isnan(racmo_AP.lat.values)

array([[False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       [ True,  True,  True, ...,  True,  True,  True]])